# LightGBM training
This is an auto-generated notebook. To reproduce these results, attach this notebook to the **automl-latest** cluster and rerun it.
- Compare trials in the [MLflow experiment](#mlflow/experiments/3877398344059750/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Navigate to the parent notebook [here](#notebook/3877398344059740) (If you launched the AutoML experiment using the Experiments UI, this link isn't very useful.)
- Clone this notebook into your project folder by selecting **File > Clone** in the notebook toolbar.

Runtime Version: _10.3.x-gpu-ml-scala2.12_

In [0]:
import mlflow
import databricks.automl_runtime

# Use MLflow to track experiments
mlflow.set_experiment("/Users/ivan.liao@validatehealth.com/databricks_automl/Beneficiary_had_at_least_one_month_in_a_Medicare_Health_Plan_patient_churn-2022_02_14-18_59")

target_col = "Beneficiary_had_at_least_one_month_in_a_Medicare_Health_Plan"

## Load Data
> **NOTE:** The dataset loaded below is a sample of the original dataset.
Stratified sampling using pyspark's [sampleBy](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameStatFunctions.sampleBy.html)
method is used to ensure that the distribution of the target column is retained.
<br/>
> Rows were sampled with a sampling fraction of **0.16227542486503857**

In [0]:
from mlflow.tracking import MlflowClient
import os
import uuid
import shutil
import pandas as pd

# Create temp directory to download input data from MLflow
input_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(input_temp_dir)


# Download the artifact and read it into a pandas DataFrame
input_client = MlflowClient()
input_data_path = input_client.download_artifacts("b6c802cace5b42e88c9688c805b8fc67", "data", input_temp_dir)

df_loaded = pd.read_parquet(os.path.join(input_data_path, "training_data"))
# Delete the temp data
shutil.rmtree(input_temp_dir)

# Preview data
df_loaded.head(5)

,ID,Sex,Birth_Year,Death_Year,Assignment_Step_Flag,Count_of_Primary_Care_Services,Beneficiary_had_a_date_of_death_prior_to_the_start_of_the_benchmark_year,Beneficiary_identifier_is_missing,Beneficiary_had_at_least_one_month_of_Part_A-only_Or_Part_B-only_Coverage,Beneficiary_does_not_reside_in_the_United_States,Beneficiary_included_in_other_Shared_Savings_Initiatives,Beneficiary_had_at_least_one_month_in_a_Medicare_Health_Plan
0,8,1,1943,NaN,True,3,False,False,False,False,False,False
1,10,2,1948,NaN,True,3,False,False,False,False,False,False
2,19,2,1944,NaN,True,1,False,False,False,False,False,False
3,20,1,1941,2018.0,True,1,False,False,False,False,False,False
4,22,1,1943,NaN,True,4,False,False,False,False,False,False


### Select supported columns
Select only the columns that are supported. This allows us to train a model that can predict on a dataset that has extra columns that are not used in training.
`["Beneficiary_included_in_other_Shared_Savings_Initiatives", "Assignment_Step_Flag"]` are dropped in the pipelines. See the Alerts tab of the AutoML Experiment page for details on why these columns are dropped.

In [0]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
supported_cols = ["Beneficiary_does_not_reside_in_the_United_States", "Count_of_Primary_Care_Services", "Beneficiary_identifier_is_missing", "Birth_Year", "Beneficiary_had_a_date_of_death_prior_to_the_start_of_the_benchmark_year", "Beneficiary_had_at_least_one_month_of_Part_A-only_Or_Part_B-only_Coverage", "Death_Year", "Sex", "ID"]
col_selector = ColumnSelector(supported_cols)

## Preprocessors

In [0]:
transformers = []

### Boolean columns
For each column, impute missing values and then convert into ones and zeros.

In [0]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder

bool_pipeline = Pipeline(steps=[
    ("cast_type", FunctionTransformer(lambda df: df.astype(object))),
    ("imputer", SimpleImputer(missing_values=None, strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

transformers.append(("boolean", bool_pipeline, ["Beneficiary_does_not_reside_in_the_United_States", "Beneficiary_had_a_date_of_death_prior_to_the_start_of_the_benchmark_year", "Beneficiary_had_at_least_one_month_of_Part_A-only_Or_Part_B-only_Coverage", "Beneficiary_identifier_is_missing"]))

### Numerical columns

Missing values for numerical columns are imputed with mean for consistency

In [0]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

numerical_pipeline = Pipeline(steps=[
    ("converter", FunctionTransformer(lambda df: df.apply(pd.to_numeric, errors="coerce"))),
    ("imputer", SimpleImputer(strategy="mean"))
])

transformers.append(("numerical", numerical_pipeline, ["Birth_Year", "Count_of_Primary_Care_Services", "ID"]))

### Categorical columns

#### Low-cardinality categoricals
Convert each low-cardinality categorical column into multiple binary columns through one-hot encoding.
For each input categorical column (string or numeric), the number of output columns is equal to the number of unique values in the input column.

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(handle_unknown="ignore")

transformers.append(("onehot", one_hot_encoder, ["Sex", "Death_Year"]))

In [0]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

### Feature standardization
Scale all feature columns to be centered around zero with unit variance.

In [0]:
from sklearn.preprocessing import StandardScaler

standardizer = StandardScaler()

## Train - Validation - Test Split
Split the input data into 3 sets:
- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters of the model)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [0]:
from sklearn.model_selection import train_test_split

split_X = df_loaded.drop([target_col], axis=1)
split_y = df_loaded[target_col]

# Split out train data
X_train, split_X_rem, y_train, split_y_rem = train_test_split(split_X, split_y, train_size=0.6, random_state=207962203, stratify=split_y)

# Split remaining data equally for validation and test
X_val, X_test, y_val, y_test = train_test_split(split_X_rem, split_y_rem, test_size=0.5, random_state=207962203, stratify=split_y_rem)

## Train classification model
- Log relevant metrics to MLflow to track runs
- All the runs are logged under [this MLflow experiment](#mlflow/experiments/3877398344059750/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Change the model parameters and re-run the training cell to log a different trial to the MLflow experiment
- To view the full list of tunable hyperparameters, check the output of the cell below

In [0]:
from lightgbm import LGBMClassifier

help(LGBMClassifier)

Help on class LGBMClassifier in module lightgbm.sklearn:

class LGBMClassifier(sklearn.base.ClassifierMixin, LGBMModel)
 |  LGBMClassifier(boosting_type: str = 'gbdt', num_leaves: int = 31, max_depth: int = -1, learning_rate: float = 0.1, n_estimators: int = 100, subsample_for_bin: int = 200000, objective: Union[str, Callable, NoneType] = None, class_weight: Union[Dict, str, NoneType] = None, min_split_gain: float = 0.0, min_child_weight: float = 0.001, min_child_samples: int = 20, subsample: float = 1.0, subsample_freq: int = 0, colsample_bytree: float = 1.0, reg_alpha: float = 0.0, reg_lambda: float = 0.0, random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None, n_jobs: int = -1, silent: Union[bool, str] = 'warn', importance_type: str = 'split', **kwargs)
 |  
 |  LightGBM classifier.
 |  
 |  Method resolution order:
 |      LGBMClassifier
 |      sklearn.base.ClassifierMixin
 |      LGBMModel
 |      sklearn.base.BaseEstimator
 |      builtins.object
 |  
 |  Met

In [0]:
import mlflow
import sklearn
from sklearn import set_config
from sklearn.pipeline import Pipeline

set_config(display="diagram")

lgbmc_classifier = LGBMClassifier(
  colsample_bytree=0.7859831258326707,
  lambda_l1=43.555814367259785,
  lambda_l2=0.13853454814718927,
  learning_rate=0.7370604369677297,
  max_bin=257,
  max_depth=4,
  min_child_samples=148,
  n_estimators=1171,
  num_leaves=187,
  path_smooth=2.0488092601119057,
  subsample=0.6669519475427949,
  random_state=207962203,
)

model = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
    ("classifier", lgbmc_classifier),
])

# Create a separate pipeline to transform the validation dataset. This is used for early stopping.
pipeline = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
])

mlflow.sklearn.autolog(disable=True)
X_val_processed = pipeline.fit_transform(X_val, y_val)

model

Pipeline Pipeline(steps=[('column_selector',
 ColumnSelector(cols=['Beneficiary_does_not_reside_in_the_United_States',
 'Count_of_Primary_Care_Services',
 'Beneficiary_identifier_is_missing',
 'Birth_Year',
 'Beneficiary_had_a_date_of_death_prior_to_the_start_of_the_benchmark_year',
 'Beneficiary_had_at_least_one_month_of_Part_A-only_Or_Part_B-only_Coverage',
 'Death_Yea...
 ('standardizer', StandardScaler()),
 ('classifier',
 LGBMClassifier(colsample_bytree=0.7859831258326707,
 lambda_l1=43.555814367259785,
 lambda_l2=0.13853454814718927,
 learning_rate=0.7370604369677297, max_bin=257,
 max_depth=4, min_child_samples=148,
 n_estimators=1171, num_leaves=187,
 path_smooth=2.0488092601119057,
 random_state=207962203,
 subsample=0.6669519475427949))]) ColumnSelector ColumnSelector(cols=['Beneficiary_does_not_reside_in_the_United_States',
 'Count_of_Primary_Care_Services',
 'Beneficiary_identifier_is_missing', 'Birth_Year',
 'Beneficiary_had_a_date_of_death_prior_to_the_start_of_the_benchmark_year',
 'Beneficiary_had_at_least_one_month_of_Part_A-only_Or_Part_B-only_Coverage',
 'Death_Year', 'Sex', 'ID']) preprocessor: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('boolean',
 Pipeline(steps=[('cast_type',
 FunctionTransformer(func= at 0x7fe962828700>)),
 ('imputer',
 SimpleImputer(missing_values=None,
 strategy='most_frequent')),
 ('onehot',
 OneHotEncoder(handle_unknown='ignore'))]),
 ['Beneficiary_does_not_reside_in_the_United_State...
 'Beneficiary_had_at_least_one_month_of_Part_A-only_Or_Part_B-only_Coverage',
 'Beneficiary_identifier_is_missing']),
 ('numerical',
 Pipeline(steps=[('converter',
 FunctionTransformer(func= at 0x7fe95f1daee0>)),
 ('imputer', SimpleImputer())]),
 ['Birth_Year',
 'Count_of_Primary_Care_Services', 'ID']),
 ('onehot',
 OneHotEncoder(handle_unknown='ignore'),
 ['Sex', 'Death_Year'])]) boolean ['Beneficiary_does_not_reside_in_the_United_States', 'Beneficiary_had_a_date_of_death_prior_to_the_start_of_the_benchmark_year', 'Beneficiary_had_at_least_one_month_of_Part_A-only_Or_Part_B-only_Coverage', 'Beneficiary_identifier_is_missing'] FunctionTransformer FunctionTransformer(func= at 0x7fe962828700>) SimpleImputer SimpleImputer(missing_values=None, strategy='most_frequent') OneHotEncoder OneHotEncoder(handle_unknown='ignore') numerical ['Birth_Year', 'Count_of_Primary_Care_Services', 'ID'] FunctionTransformer FunctionTransformer(func= at 0x7fe95f1daee0>) SimpleImputer SimpleImputer() onehot ['Sex', 'Death_Year'] OneHotEncoder OneHotEncoder(handle_unknown='ignore') remainder [['Beneficiary_does_not_reside_in_the_United_States', 'Count_of_Primary_Care_Services', 'Beneficiary_identifier_is_missing', 'Birth_Year', 'Beneficiary_had_a_date_of_death_prior_to_the_start_of_the_benchmark_year', 'Beneficiary_had_at_least_one_month_of_Part_A-only_Or_Part_B-only_Coverage', 'Death_Year', 'Sex', 'ID']] passthrough passthrough StandardScaler StandardScaler() LGBMClassifier LGBMClassifier(colsample_bytree=0.7859831258326707,
 lambda_l1=43.555814367259785, lambda_l2=0.13853454814718927,
 learning_rate=0.7370604369677297, max_bin=257, max_depth=4,
 min_child_samples=148, n_estimators=1171, num_leaves=187,
 path_smooth=2.0488092601119057, random_state=207962203,
 subsample=0.6669519475427949)

In [0]:
# Enable automatic logging of input samples, metrics, parameters, and models
mlflow.sklearn.autolog(log_input_examples=True, silent=True)

with mlflow.start_run(run_name="lightgbm") as mlflow_run:
    model.fit(X_train, y_train, classifier__early_stopping_rounds=5, classifier__eval_set=[(X_val_processed,y_val)], classifier__verbose=False)
    
    # Training metrics are logged by MLflow autologging
    # Log metrics for the validation set
    lgbmc_val_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_val, y_val, prefix="val_")

    # Log metrics for the test set
    lgbmc_test_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_test, y_test, prefix="test_")

    # Display the logged metrics
    lgbmc_val_metrics = {k.replace("val_", ""): v for k, v in lgbmc_val_metrics.items()}
    lgbmc_test_metrics = {k.replace("test_", ""): v for k, v in lgbmc_test_metrics.items()}
    display(pd.DataFrame([lgbmc_val_metrics, lgbmc_test_metrics], index=["validation", "test"]))

/databricks/python/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/databricks/python/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

[LightGBM] [Warning] lambda_l1 is set=43.555814367259785, reg_alpha=0.0 will be ignored. Current value: lambda_l1=43.555814367259785
[LightGBM] [Warning] lambda_l2 is set=0.13853454814718927, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.13853454814718927


/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision_score,recall_score,f1_score,accuracy_score,log_loss,roc_auc_score,score
validation,0.855123,0.924729,0.888565,0.924729,0.265620,0.566829,0.924729
test,0.855579,0.924975,0.888925,0.924975,0.264462,0.593402,0.924975


In [0]:
# Patch requisite packages to the model environment YAML for model serving
import os
import shutil
import uuid
import yaml

None

import lightgbm
from mlflow.tracking import MlflowClient

lgbmc_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], str(uuid.uuid4())[:8])
os.makedirs(lgbmc_temp_dir)
lgbmc_client = MlflowClient()
lgbmc_model_env_path = lgbmc_client.download_artifacts(mlflow_run.info.run_id, "model/conda.yaml", lgbmc_temp_dir)
lgbmc_model_env_str = open(lgbmc_model_env_path)
lgbmc_parsed_model_env_str = yaml.load(lgbmc_model_env_str, Loader=yaml.FullLoader)

lgbmc_parsed_model_env_str["dependencies"][-1]["pip"].append(f"lightgbm=={lightgbm.__version__}")

with open(lgbmc_model_env_path, "w") as f:
  f.write(yaml.dump(lgbmc_parsed_model_env_str))
lgbmc_client.log_artifact(run_id=mlflow_run.info.run_id, local_path=lgbmc_model_env_path, artifact_path="model")
shutil.rmtree(lgbmc_temp_dir)

## Feature importance

SHAP is a game-theoretic approach to explain machine learning models, providing a summary plot
of the relationship between features and model output. Features are ranked in descending order of
importance, and impact/color describe the correlation between the feature and the target variable.
- Generating SHAP feature importance is a very memory intensive operation, so to ensure that AutoML can run trials without
  running out of memory, we disable SHAP by default.<br />
  You can set the flag defined below to `shap_enabled = True` and re-run this notebook to see the SHAP plots.
- To reduce the computational overhead of each trial, a single example is sampled from the validation set to explain.<br />
  For more thorough results, increase the sample size of explanations, or provide your own examples to explain.
- SHAP cannot explain models using data with nulls; if your dataset has any, both the background data and
  examples to explain will be imputed using the mode (most frequent values). This affects the computed
  SHAP values, as the imputed samples may not match the actual data distribution.

For more information on how to read Shapley values, see the [SHAP documentation](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html).

In [0]:
# Set this flag to True and re-run the notebook to see the SHAP plots
shap_enabled = False

In [0]:
if shap_enabled:
    from shap import KernelExplainer, summary_plot
    # SHAP cannot explain models using data with nulls.
    # To enable SHAP to succeed, both the background data and examples to explain are imputed with the mode (most frequent values).
    mode = X_train.mode().iloc[0]

    # Sample background data for SHAP Explainer. Increase the sample size to reduce variance.
    train_sample = X_train.sample(n=min(100, len(X_train.index))).fillna(mode)

    # Sample a single example from the validation set to explain. Increase the sample size and rerun for more thorough results.
    example = X_val.sample(n=1).fillna(mode)

    # Use Kernel SHAP to explain feature importance on the example from the validation set.
    predict = lambda x: model.predict(pd.DataFrame(x, columns=X_train.columns))
    explainer = KernelExplainer(predict, train_sample, link="identity")
    shap_values = explainer.shap_values(example, l1_reg=False)
    summary_plot(shap_values, example, class_names=model.classes_)

## Inference
[The MLflow Model Registry](https://docs.databricks.com/applications/mlflow/model-registry.html) is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance. The snippets below show how to add the model trained in this notebook to the model registry and to retrieve it later for inference.

> **NOTE:** The `model_uri` for the model already trained in this notebook can be found in the cell below

### Register to Model Registry
```
model_name = "Example"

model_uri = f"runs:/{ mlflow_run.info.run_id }/model"
registered_model_version = mlflow.register_model(model_uri, model_name)
```

### Load from Model Registry
```
model_name = "Example"
model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
model.predict(input_X)
```

### Load model without registering
```
model_uri = f"runs:/{ mlflow_run.info.run_id }/model"

model = mlflow.pyfunc.load_model(model_uri)
model.predict(input_X)
```

In [0]:
# model_uri for the generated model
print(f"runs:/{ mlflow_run.info.run_id }/model")

runs:/3af8008d41c044e0b7a2bdf946b0223c/model
